In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [4]:
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	# Splitting the text into sentences using
	# delimiters like '.', '?', and '!'
	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [9]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [10]:
# Train the model
model.fit(X, y, epochs=500, verbose=1)


Epoch 1/500
51/51 [==============================] - 6s 73ms/step - loss: 6.2008 - accuracy: 0.0405
Epoch 2/500
51/51 [==============================] - 3s 50ms/step - loss: 5.7866 - accuracy: 0.0510
Epoch 3/500
51/51 [==============================] - 3s 58ms/step - loss: 5.7285 - accuracy: 0.0522
Epoch 4/500
51/51 [==============================] - 3s 49ms/step - loss: 5.7045 - accuracy: 0.0565
Epoch 5/500
51/51 [==============================] - 3s 63ms/step - loss: 5.6736 - accuracy: 0.0565
Epoch 6/500
51/51 [==============================] - 3s 60ms/step - loss: 5.6346 - accuracy: 0.0565
Epoch 7/500
51/51 [==============================] - 3s 50ms/step - loss: 5.5876 - accuracy: 0.0534
Epoch 8/500
51/51 [==============================] - 3s 49ms/step - loss: 5.5444 - accuracy: 0.0651
Epoch 9/500
51/51 [==============================] - 3s 50ms/step - loss: 5.4857 - accuracy: 0.0651
Epoch 10/500
51/51 [==============================] - 4s 74ms/step - loss: 5.4245 - accuracy: 0.0737

In [15]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 10

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)



1/1 [==============================] - 0s 47ms/step
Next predicted words: Pizza have different  become a symbol of comfort happiness and celebration and its
